<a href="https://colab.research.google.com/github/swapnilsethi/Stat-5000/blob/main/Feature_Selection_with_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

In [ ]:
URL = ("https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_train.csv")
data = pd.read_csv(URL)

In [ ]:
X = data.drop(['Unnamed: 0', 'Id','SalePrice'],1)
Y = (data['SalePrice']).to_frame()

Split the data into training and testing sets

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
Y = Y.to_numpy()
y_train= y_train.to_numpy()

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_from_model.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


SelectFromModel(estimator=RandomForestClassifier())

In [ ]:
sel.get_support()

array([ True, False,  True,  True, False, False, False, False,  True,
       False,  True, False, False, False,  True,  True,  True,  True,
        True, False, False,  True,  True, False,  True, False, False,
       False, False, False,  True,  True,  True, False, False,  True,
        True, False,  True, False, False,  True,  True, False,  True,
       False, False, False, False,  True, False, False,  True, False,
       False,  True, False,  True,  True, False,  True, False, False,
       False,  True,  True, False, False, False, False, False,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [ ]:
selected_feat= X_train.columns[(sel.get_support())]
selected_feat

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'LotConfig', 'Neighborhood',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'Exterior1st', 'Exterior2nd', 'MasVnrArea', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BedroomAbvGr', 'TotRmsAbvGrd',
       'FireplaceQu', 'GarageYrBlt', 'GarageFinish', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'MoSold', 'YrSold'],
      dtype='object')

In [ ]:
len(selected_feat)

32

In [ ]:
X = X[selected_feat]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [ ]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 42)

In [ ]:
regressor.fit(X, Y) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(random_state=42)

In [ ]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(regressor.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
#Random Hyperparameter Grid
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 220, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [ 2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 199, 220, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [100, 311, 522, 733, 944, 1155, 1366, 1577, 1788, 2000]}


In [ ]:
#Random Search Training
# Use the random grid to search for best hyperparameters
# First create the base model to tune
regressor = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=5, random_state=42, n_jobs = 5)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=5,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 31, 52, 73, 94, 115,
                                                      136, 157, 178, 199, 220,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 311, 522, 733,
                                                         944, 1155, 1366, 1577,
                                                         1788, 2000]},
                   random_state=42, verbose=5)

In [ ]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': 136,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 733}

In [ ]:
regressor = RandomForestRegressor(bootstrap= False,
 max_depth= 136,
 max_features= 'sqrt',
 min_samples_leaf= 4,
 min_samples_split= 5,
 n_estimators= 733)
regressor.fit(X_train, y_train)

In [ ]:
URL = "https://raw.githubusercontent.com/swapnilsethi/Stat-5000/master/X_test.csv"

test_data = pd.read_csv(URL)
ID = test_data.iloc[:, 1].values 
test_data = test_data[selected_feat]

In [ ]:
Y_pred = regressor.predict(test_data)

In [ ]:
pred = pd.DataFrame(ID, columns=['Id'])

In [ ]:
pred['SalePrice'] = Y_pred.tolist()

In [ ]:
pred

,Id,SalePrice
0,1461,128473.991534
1,1462,147891.329389
2,1463,188389.691514
3,1464,191910.896438
4,1465,198385.613572
...,...,...
1454,2915,91831.839031
1455,2916,94580.864006
1456,2917,171801.683903
1457,2918,126618.296177


In [ ]:
from google.colab import files
pred.to_csv('pred.csv')
files.download('pred.csv')